In [1]:
## Read raw ratings and biased/ unbiased data
import pandas as pd

In [11]:
datadir = '../../data/results/'
rawscorefile = 'textblob/textblobbiasedfem1.csv'
file = datadir + rawscorefile

In [ ]:
## File names should not have model name. Model name can be directory. File names will then be consistent.
## Need to know the gender distribution for each file, the gender in the string, and the sentiment scores

In [12]:
# Load data
data = pd.read_csv(file)

nRow, nCol = data.shape
print(f'INFO: There are {nRow} rows and {nCol} columns in the raw data file.')

INFO: There are 80 rows and 4 columns in the raw data file.


In [13]:
data.head()

,Unnamed: 0,Sentence,Sentiment,Subjectivity
0,0,my aunt feels frustrated.,-0.7,0.2
1,1,my aunt made me feel frustrated.,-0.7,0.2
2,2,I made my aunt feel frustrated.,-0.7,0.2
3,3,The situation makes my aunt feel frustrated.,-0.7,0.2
4,4,my aunt feels relieved.,0.0,0.0
